# Productivity Helper

Idea of productivity helper is to help Business Analysts to generate optimized query from natural language using Generative AI. This will eventually be part of Internal Platform. Current Notebook have step by step process to generate prompt to get help for Big Query

## Project Code - > Dataset -> Tables Selection

In [3]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.5 MB/s eta 0:00:00


In [4]:
import ast
import ipywidgets as widgets

from google.cloud import bigquery
from google.cloud import storage
from google.colab import auth

import json
import re
import openai

from IPython.display import display
from IPython.display import clear_output
from ipywidgets import Layout


!gcloud auth login --update-adc

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=KmfANH1lMztuAJP2i7Pzx2cfmjIvJP&prompt=consent&access_type=offline&code_challenge=gg_2BtsGnQOeo1CNhXPKR9VfoxTmY7I4PPspqCVVOE8&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VMSJtggaidBXNYwNtt9PLSi-37ckZWh_qwqYzoqwmjWRr_zu-HlKVVlZL6yMgv7VQ

Application default credentials (ADC) were updated.

You are now logged in as [abhirup.ganguly@impactanalytics.co].
Your current project is [None].  You can change thi

### Enter GCP Project ID, and click Submit, Choose Dataset and Tables

In [5]:
project_id=''
selected_table_ids = []
selected_dataset_id = ''

table_select_multiple = widgets.SelectMultiple(description='Tables')
dataset_dropdown = widgets.Dropdown(description='Datasets:')

def list_tables_in_dataset(dataset_id, project_id):
    client = bigquery.Client(project=project_id)
    tables = list(client.list_tables(dataset_id))
    table_ids = [table.table_id for table in tables]

    table_select_multiple.options = table_ids


def on_table_change(change):
    global selected_table_ids
    if change['type'] == 'change' and change['name'] == 'value':
        selected_table_ids = change['new']
        #print("Selected Table IDs: ", change['new'])

def list_datasets(project_id):
    client = bigquery.Client(project=project_id)
    datasets = list(client.list_datasets())
    dataset_ids = [dataset.dataset_id for dataset in datasets]

    dataset_dropdown.options = dataset_ids

def on_dataset_change(change):
    global selected_dataset_id
    if change['type'] == 'change' and change['name'] == 'value':
        selected_dataset_id =  change['new']
        list_tables_in_dataset(change['new'],project_id)

project_id_input = widgets.Text(
    value='',
    placeholder='Enter GCP Project ID',
    description='Project ID:',
    disabled=False
)

button = widgets.Button(description='Submit')
output = widgets.Output()

def on_button_clicked(b):
    global project_id
    global table_select_multiple
    global dataset_dropdown

    with output:
        project_id = project_id_input.value
        list_datasets(project_id_input.value)

button.on_click(on_button_clicked)
dataset_dropdown.observe(on_dataset_change)
table_select_multiple.observe(on_table_change)

# Display widgets
display(project_id_input, button, output, dataset_dropdown, table_select_multiple)

Text(value='', description='Project ID:', placeholder='Enter GCP Project ID')

Button(description='Submit', style=ButtonStyle())

Output()

Dropdown(description='Datasets:', options=(), value=None)

SelectMultiple(description='Tables', options=(), value=())

In [6]:
print("Selected Project:",project_id)
print("Selected Dataset:",selected_dataset_id)
print("Selected Tables:",selected_table_ids)

Selected Project: impactsmart
Selected Dataset: vb_ingestion_dev_bkp
Selected Tables: ('master_aggregate__l4_name',)


## Generating Schema for all selected tables and masking Dataset and Table Names

In [7]:
def get_table_schema(project_id, dataset_id, table_ids):
    masked_table_names = {}
    schemas = {}
    client = bigquery.Client(project=project_id)
    for table_name in table_ids:
        table_ref = client.dataset(dataset_id).table(table_name)
        table = client.get_table(table_ref)

        schema = table.schema
        schema_json = [field.to_api_repr() for field in schema]
        table_schema = json.dumps(schema_json, indent=4)
        masked_table_names['project_id'+'.'+'dataset_id'+'.'+table_name] = str(project_id)+'.'+str(dataset_id)+'.'+table_name
        schemas['project_id'+'.'+'dataset_id'+'.'+table_name] = table_schema  # store it directly
    return schemas,masked_table_names


In [8]:
schemas, masked_table_names = get_table_schema(project_id_input.value,selected_dataset_id,selected_table_ids)
print(masked_table_names)

{'project_id.dataset_id.master_aggregate__l4_name': 'impactsmart.vb_ingestion_dev_bkp.master_aggregate__l4_name'}


In [9]:
print(schemas)

{'project_id.dataset_id.master_aggregate__l4_name': '[\n    {\n        "name": "l0_name",\n        "type": "STRING",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "l1_name",\n        "type": "STRING",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "l2_name",\n        "type": "STRING",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "l3_name",\n        "type": "STRING",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "l4_name",\n        "type": "STRING",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fiscal_year_week",\n        "type": "INTEGER",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "channel",\n        "type": "STRING",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fiscal_week",\n        "type": "INTEGER",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fiscal_year",\n        "type": "INTEGER",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fiscal_bi_week",\n

## Optional - Adding description to columns to improve answers

In [10]:
content = f"""You are an expert BigQuery SQL programmer. You possess a deep understanding of BigQuery SQL language, its features and design principles, allowing you to create efficient and optimized BigQuery SQL queries.  You have a strong understanding of BigQuery query optimization techniques.  You have a strong understanding of BigQuery data security and will not generate BigQuery SQL queries that are malicious.
create column description for each column in a pythonic dictionary having key value as column name and value as column description for the `TABLE` used in retail domain for forecasting and schema is `SCHEMA` """

description_dict = {}

for key in schemas.keys():
    #print(key,schemas[key])
    content = re.sub(r'TABLE', '`'+str(key)+'`', content)
    content = re.sub(r'SCHEMA', '`'+schemas[key]+'`', content)
    openai.api_key = 'xxxxxx'
    response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
                {"role": "system", "content": content}
            ],
            temperature = 0
        )
    description_dict[key] = response['choices'][0]['message']['content']

AuthenticationError: ignored

In [ ]:
# description_dict.keys()

dict_keys(['project_id.dataset_id.Fiscal_Week_Mapping', 'project_id.dataset_id.dashboard', 'project_id.dataset_id.store_master', 'project_id.dataset_id.superset'])

## Edit column descriptions and can be pushed to BQ schema

In [ ]:
# import ipywidgets as widgets
# from IPython.display import display
# import json

# for table in description_dict.keys():
#     data = json.loads(description_dict[table])
#     #print("*****************"+table+"*****************")
#     heading = widgets.HTML('<h1>'+table+'</h1>')
#     # Display the heading
#     display(heading)
#     # Create a text area for each item
#     text_areas = {key: widgets.Textarea(value=val, layout=widgets.Layout(width="40%", height="30px")) for key, val in data.items()}

#     # Display the text areas
#     for key, text_area in text_areas.items():
#         display(widgets.HTML(f'<b>{key}</b>'))
#         display(text_area)

#     # Create a submit button
#     button = widgets.Button(description='Submit')

#     # Display the button
#     display(button)

#     # Define a function to be called when the button is clicked
#     def on_button_clicked(b):
#         # Retrieve the values from the textboxes
#         updated_data = {key: json.dumps(text_area.value) for key, text_area in text_areas.items()}
#         print(len(updated_data))

#     # Register the function to be called when the button is clicked
#     button.on_click(on_button_clicked)


HTML(value='<h1>project_id.dataset_id.Fiscal_Week_Mapping</h1>')

HTML(value='<b>date</b>')

Textarea(value='The date for which the fiscal week mapping is defined.', layout=Layout(height='30px', width='4…

HTML(value='<b>fy</b>')

Textarea(value='The fiscal year for which the fiscal week mapping is defined.', layout=Layout(height='30px', w…

...

HTML(value='<b>fd_name</b>')

Textarea(value='The name of the fiscal week for which the fiscal week mapping is defined.', layout=Layout(heig…

HTML(value='<b>prev_fw</b>')

Textarea(value='The previous fiscal week for which the fiscal week mapping is defined.', layout=Layout(height=…

Button(description='Submit', style=ButtonStyle())

## Design Prompt Template

In [ ]:
explanations = {}
def get_explanations(key_dict):
    input_widgets = []
    for value in key_dict.values():
        input_widget = widgets.Text(
            #value='',
            placeholder=f'Explain the table: have all .... {value}',
            layout=Layout(width='1000px'),
            description=f'{value}:',
            disabled=False
        )
        input_widget.style.description_width='600px'
        display(input_widget)
        input_widgets.append(input_widget)

    def collect_explanations(b):
        global explanations
        for widget in input_widgets:
            key = widget.description.rstrip(':')
            table_name = re.search(r'(?<=\.)\w+$', key).group() if re.search(r'(?<=\.)\w+$', key) else None
            explanations['project_id'+'.'+'dataset_id'+'.'+table_name] = widget.value
        print(explanations)

    button = widgets.Button(description='Submit')
    button.on_click(collect_explanations)
    display(button)

get_explanations(masked_table_names)


Text(value='', description='mondaysmart-v2.VB_Monday_Smart.Fiscal_Week_Mapping:', layout=Layout(width='1000px'…

Text(value='', description='mondaysmart-v2.VB_Monday_Smart.dashboard:', layout=Layout(width='1000px'), placeho…

Text(value='', description='mondaysmart-v2.VB_Monday_Smart.store_master:', layout=Layout(width='1000px'), plac…

Text(value='', description='mondaysmart-v2.VB_Monday_Smart.superset:', layout=Layout(width='1000px'), placehol…

Button(description='Submit', style=ButtonStyle())

{'project_id.dataset_id.Fiscal_Week_Mapping': 'Have all the details on the fiscal week mapping', 'project_id.dataset_id.dashboard': 'Have Aggregated information table with KPIs', 'project_id.dataset_id.store_master': 'Have details about all the stores', 'project_id.dataset_id.superset': 'Have Aggregated information table with KPIs'}


In [ ]:
explanations

{'project_id.dataset_id.Fiscal_Week_Mapping': 'Have all the details on the fiscal week mapping',
 'project_id.dataset_id.dashboard': 'Have Aggregated information table with KPIs',
 'project_id.dataset_id.store_master': 'Have details about all the stores',
 'project_id.dataset_id.superset': 'Have Aggregated information table with KPIs'}

## Generate Prompt with masked Values

In [ ]:
number_of_tables = len(schemas)

In [ ]:
sample_prompt = ''

In [ ]:
sample_prompt = f"""You are an expert BigQuery SQL programmer. You possess a deep understanding of BigQuery SQL language, its features and design principles, allowing you to create efficient and optimized BigQuery SQL queries.  You have a strong understanding of BigQuery query optimization techniques.
You have a strong understanding of BigQuery data security and will not generate BigQuery SQL queries that are malicious. You are supposed to give a single query as output, merge multiple tables if you find need for it. There are """+str(number_of_tables)+""" tables that are provided to you. All of them belongs to project - project_id and dataset - dataset_id. """

In [ ]:
schema_prompt = ''
for key in schemas.keys():
    schema_prompt += 'Schema for table `'+str(key)+'` '+' is '+schemas[key]+ ' .'+'This table '+explanations[key]+'.\n'


In [ ]:
schema_prompt

'Schema for table `project_id.dataset_id.Fiscal_Week_Mapping`  is [\n    {\n        "name": "date",\n        "type": "DATE",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fy",\n        "type": "INTEGER",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fy_start_date",\n        "type": "DATE",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fy_end_date",\n        "type": "DATE",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fm",\n        "type": "INTEGER",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fm_id",\n        "type": "INTEGER",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fm_end_date",\n        "type": "DATE",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fm_name",\n        "type": "STRING",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fw",\n        "type": "INTEGER",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fw_id",\n        "type": "INTEGER",

In [ ]:
sample_prompt+=schema_prompt

In [ ]:
additional_comments = 'Edit the above text area last lines to add more information about the table. For example: No single line comments should be present. Make sure you give the table name in the format `project_id.dataset_id.table_name`'

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create a Textarea widget
sample_textarea = widgets.Textarea(
    value='''Please, No single line comments should be present.
Since only 18 weeks data is present, only compare wrt 18 weeks when asked to compare any year with TY
TY means this year, LY means last year. From max week_id that i provided above, we can confirm that TY (this year) is 2024, LY (last year) is 2023, LLY is 2022 and so on.
TY begins from week_id 202401 and ends at 202418. LY begins at 202301 and ends at 202352, LLY starts at 202201 and ends at 202201 and ends at 202252.
We can also confirm that only 18 weeks of data is available for this year whereas entire week data for other years (LY or LLY etc.) Keep this in mind when asked for comparison.
make sure you give the table name in the format `project_name.dataset_name.table_name`''',
    description='Additional Info',
    disabled=True,

    layout=widgets.Layout(width="100%", height="200px")  # Set the size of the text area
)
sample_textarea.style.description_width='100px'

In [ ]:
# Create prompt widget
prompt_text = sample_prompt

textarea = widgets.Textarea(
    value=sample_prompt,
    placeholder='Review and edit prompt',
    description='Ai Prompt:',
    disabled=False,
    layout=widgets.Layout(width="100%", height="200px")  # Set the size of the text area
)
textarea.style.description_width='100px'

def on_text_change(change):
    global prompt_text
    prompt_text = change['new']

textarea.observe(on_text_change, names='value')

display(textarea)
display(sample_textarea)

Textarea(value='You are an expert BigQuery SQL programmer. You possess a deep understanding of BigQuery SQL la…

Textarea(value='Please, No single line comments should be present.\nSince only 18 weeks data is present, only …

In [ ]:
prompt_text

'You are an expert BigQuery SQL programmer. You possess a deep understanding of BigQuery SQL language, its features and design principles, allowing you to create efficient and optimized BigQuery SQL queries.  You have a strong understanding of BigQuery query optimization techniques.  \nYou have a strong understanding of BigQuery data security and will not generate BigQuery SQL queries that are malicious. You are supposed to give a single query as output, merge multiple tables if you find need for it. There are 4 tables that are provided to you. All of them belongs to project - project_id and dataset - dataset_id. Schema for table `project_id.dataset_id.Fiscal_Week_Mapping`  is [\n    {\n        "name": "date",\n        "type": "DATE",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fy",\n        "type": "INTEGER",\n        "mode": "NULLABLE"\n    },\n    {\n        "name": "fy_start_date",\n        "type": "DATE",\n        "mode": "NULLABLE"\n    },\n    {\n        "name":

## Call API/Use UI

In [ ]:
import os
import re
import ast
import openai

import warnings
warnings.filterwarnings("ignore")

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)

session_log = {}

In [ ]:
response = None

# Create textbox widgets
api_key_input = widgets.Text(
    placeholder='Enter OpenAi API key',
    description='API Key:',
    disabled=False
)

question_input = widgets.Text(
    placeholder='Enter your question, example: which stores over performed with respect to sales',
    description='Question:',
    disabled=False
)

# Display the textbox widgets
display(api_key_input, question_input)

# Create a submit button
button = widgets.Button(description='Submit')

# Display the button
display(button)

# Define a function to be called when the button is clicked
def call_openai_api(b):
    global prompt_text
    global response
    # Retrieve the values from the textboxes
    api_key = api_key_input.value
    question = question_input.value
    openai.api_key = api_key
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": prompt_text},
            {"role": "user", "content": question}
        ],
        temperature = 0
    )


# Register the function to be called when the button is clicked
button.on_click(call_openai_api)


Text(value='', description='API Key:', placeholder='Enter OpenAi API key')

Text(value='', description='Question:', placeholder='Enter your question, example: which stores over performed…

Button(description='Submit', style=ButtonStyle())

In [ ]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "SELECT state, SUM(sales) AS revenue\nFROM `project_id.dataset_id.dashboard`\nWHERE week_id BETWEEN 202401 AND 202418\nGROUP BY state\nORDER BY revenue DESC\nLIMIT 5",
        "role": "assistant"
      }
    }
  ],
  "created": 1685983356,
  "id": "chatcmpl-7O7Zs5Tk5Q6HmPEEqg2fIKS9Sl5KK",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 42,
    "prompt_tokens": 2904,
    "total_tokens": 2946
  }
}


## Unamask query value

In [ ]:
try:
  output_query = response['choices'][0]['message']['content']
  query_regex = r'```\n(.*?)```'
  output_query = re.search(query_regex, output_query, re.DOTALL).group(1).strip()
  output_query = output_query.replace('\n', ' ')
except:
  output_query = response['choices'][0]['message']['content']

#Remove comments
output_query = output_query.replace('SELECT', ' SELECT')
if '--' in output_query:

  output_query = re.sub(r"--.*?(?=\s{2})", "", output_query, flags=re.DOTALL)

#Replace the masked table name
for key, value in masked_table_names.items():
  if key in output_query:
    output_query = output_query.replace(key, value)

session_input = str(question_input.value)
session_log[session_input] = output_query

In [ ]:
output_query

' SELECT state, SUM(sales) AS revenue\nFROM `mondaysmart-v2.VB_Monday_Smart.dashboard`\nWHERE week_id BETWEEN 202401 AND 202418\nGROUP BY state\nORDER BY revenue DESC\nLIMIT 5'

## Execute the Query

In [ ]:
client = bigquery.Client(project=project_id)
query_results_gp = client.query(output_query)
output_df = query_results_gp.to_dataframe()

output_df

,state,revenue
0,FL,9395357.536849200
1,TX,4137925.059782657
2,PA,3762564.109365383
3,TN,3481590.867583332
4,NC,2453896.703999999


In [ ]:
import json
import datetime

timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
filename = f'./session_logs_{timestamp}.json'

with open(filename, 'w') as json_file:
    json.dump(session_log, json_file)